In [5]:
import pandas as pd
import glob
import xml.etree.ElementTree as ET
from google.cloud import vision
import io
import time
from pathlib import Path
import os

from tqdm import tqdm
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join('.', 'gcp/googlecreds.json')

text_list = []


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            class_name = member[0].text
            if class_name == 'keyvalue':
                value = (root.find('filename').text,
                         int(root.find('size')[0].text),
                         int(root.find('size')[1].text),
                         member[0].text,
                         int(member[4][0].text),
                         int(member[4][1].text),
                         int(member[4][2].text),
                         int(member[4][3].text)
                         )
                xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def getTextGroupData(imagePath):
    vertexList = []
    client = vision.ImageAnnotatorClient()
    with io.open(imagePath, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    bool = False
    complete_text = ''
    for text in texts:
        vertices = text.bounding_poly.vertices
        if bool:
            vertexList.append((text.description, vertices))
        else:
            complete_text = text.description
            bool = True

    if response.error.message:
        raise Exception(
            '{}\n'.format(response.error.message))
    return vertexList



def get_text_single_bbox(vertexList, bbox):
    text = ''
    boundingBox = bbox

    for vertexText in vertexList:
        vertex = vertexText[1]
        x1 = vertex[0].x
        y1 = vertex[0].y
        x2 = vertex[2].x
        y2 = vertex[2].y
        height = y2 - y1
        width = x2 - x1

        if (not (max(x1, boundingBox[0]) + width / 3 > min(x2, boundingBox[2]))) and \
                (not (max(y1, boundingBox[1]) + height / 3 > min(y2, boundingBox[3]))):
            text += vertexText[0] + ' '
    text_list.append(text)
    return text


In [7]:
def main():
    base_path = 'C:/Users/KartikayGupta/NER/inference/infer_set/'
    xml_df = xml_to_csv(base_path)
    xml_df.to_excel('AN+BL_annotations.xlsx')
    xml_df = pd.read_excel('AN+BL_annotations.xlsx', engine='openpyxl')

    final_df = pd.DataFrame()
    files = xml_df['filename'].unique()
    i = 0
    for file in tqdm(files):
        tx_ls = []
        start = time.time()
        df = xml_df[xml_df['filename'] == file]
        if not os.path.exists(base_path + file):
            continue
        vertexList = getTextGroupData(base_path + file)
        for index, row in df.iterrows():
            bbox = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
            text = get_text_single_bbox(vertexList, bbox)
            tx_ls.append(text)
            final_df.loc[i, 'filename'] = file
            final_df.loc[i, 'text'] = text
            final_df.loc[i, 'label'] = df.loc[index, 'class']
            i = i+1
        file_nm = Path(file).stem
        with open(f'C:/Users/KartikayGupta/NER/inference/res/{file_nm}.txt', 'w' , encoding="utf-8") as f:
            for line in tx_ls:
                f.write(line)
                f.write('\n')       
        print(file + ' --> done')
        print(time.time()-start)
    #final_df.to_excel('AN+BL-not-seperated-keyvalues.xlsx')


if __name__ == '__main__':
    main()


  0%|          | 1/1271 [00:05<1:59:08,  5.63s/it]

1000176487-1286007669-ArrivalNotice-5-7-11h57m28s-1.png --> done
5.6286351680755615


  0%|          | 2/1271 [00:10<1:48:30,  5.13s/it]

1000472365-1000464494-REVISEDARRIVALNOTICE-FREIGHTINVOICEHBLVNOAK2103974MBLONEYSGNB82499600-1.png --> done
4.781420707702637


  0%|          | 3/1271 [00:16<2:00:38,  5.71s/it]

...

test_CI - Copy (3).png --> done
4.701979875564575


100%|█████████▉| 1270/1271 [1:47:18<00:04,  4.77s/it]

test_CI - Copy.png --> done
4.304287433624268


100%|██████████| 1271/1271 [1:47:22<00:00,  5.07s/it]

test_CI.png --> done
4.507946491241455


In [17]:
print(len(text_list))

1113


In [18]:
with open('final_data.txt', 'w' , encoding="utf-8") as f:
    for line in text_list:
        f.write(line)
        f.write('\n')